# Explore eo-learn package 

In [ ]:
# ML libs
import pandas as pd
import numpy as np
from aenum import MultiValueEnum
import geopandas as gpd

import datetime
# File manipulation
import os
from urllib.request import urlopen
import json
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
import rasterio
import rasterio.plot as rplt

# Graphic
import matplotlib.pyplot as plt
from matplotlib.colors import BoundaryNorm
import matplotlib.cm as cmap
import matplotlib.pyplot as plt
import plotly.express as px
from sentinelhub import BBox, CRS
import shapely

# Environment variables
datasets_dir = os.environ.get('DATA_DIR_LOCAL')

In [ ]:
one_view_dir = "RPG/S2A_MSIL2A_20240412T105621_N0510_R094_T30TXP_20240412T165247.SAFE"
one_view_path = os.path.join(datasets_dir,one_view_dir)

# Accessing the bounding-box info of the satellite image

In [ ]:
one_view_path = os.path.join(one_view_path,os.listdir(one_view_path)[0])

In [ ]:
os.listdir(one_view_path)

In [ ]:
MTD_MSIL2A_file = os.path.join(one_view_path, 'MTD_MSIL2A.xml')
with open(MTD_MSIL2A_file) as fp:
    MTD_MSIL2A_soup = BeautifulSoup(fp,'xml')

In [ ]:
MTD_MSIL2A_coord_list = MTD_MSIL2A_soup.find('EXT_POS_LIST').text.split()

In [ ]:
coord_array = np.array(MTD_MSIL2A_coord_list).reshape(int(len(MTD_MSIL2A_coord_list)/2), 2)

## Create geopandas dataframe with image bounds

In [ ]:
#TODO compute local_CRS
local_CRS = '32630'

In [ ]:
img_gdf = gpd.GeoDataFrame({"id": [0], "geometry": [shapely.geometry.Polygon([[lon, lat] for lat, lon in coord_array])]}, crs="EPSG:4326").to_crs(local_CRS)
img_gdf['geometry']

## Plot sentinel image, pays-basque shape, bboxes of selected area

In [ ]:
left, bottom, right, top = img_gdf['geometry'][0].bounds

In [ ]:
img_path = os.path.join(one_view_path, 'GRANULE/L2A_T30TXP_A045994_20240412T110532/IMG_DATA/R10m/T30TXP_20240412T105621_TCI_10m.jp2')

# Reading the jp2s with rasterio
with rasterio.open(img_path) as f:
    sentinel_img=np.array(f.read())

In [ ]:
rplt.show(sentinel_img) 

In [ ]:
shape = img_gdf['geometry'][0]
fig_size = 10
fig, ax = plt.subplots(figsize=(fig_size, fig_size))
ax.set_title(f"Selected {size_of_selected_area}x{size_of_selected_area} tiles from Pays basque", fontsize=0.2*fig_size)
zone_covered.plot(ax=ax, facecolor="w", edgecolor="b", alpha=0.8)
img_gdf['geometry'].plot(ax=ax, alpha=0.1)
bbox_gdf[bbox_gdf.index.isin(patch_ids)].plot(ax=ax, facecolor="w", edgecolor="r", alpha=0.5)
#plt.imshow(sentinel_img, extent=(left, right, bottom, top), alpha=0.5);
rplt.show(sentinel_img, ax=ax) 

## Create eopatch from sentinel image

In [ ]:
eopatch = EOPatch(bbox=BBox(bbox=shape.bounds, crs=CRS('32630')))

# If the image has 3 channels (RGB), add it as a timeless feature
if sentinel_img.ndim == 3:
    eopatch.data_timeless['BANDS'] = sentinel_img
# If the image is grayscale (2D), add an extra channel dimension
else:
    eopatch.data_timeless['BANDS'] = sentinel_img[..., np.newaxis]

In [ ]:
eopatch

In [ ]:
plt.imshow(eopatch.data_timeless["BANDS"], extent=(left, right, bottom, top));

## Create sub patches

In [ ]:
small_patch = create_subpatch(eopatch, bbox_list[20])
small_patch

In [ ]:
left, bottom, right, top = small_patch.bbox

In [ ]:
bbox_list[20].get_polygon()

In [ ]:
gpd.GeoDataFrame(bbox_list[36])
small_box_gdf = gpd.GeoDataFrame({"id": [0], "geometry": [shapely.geometry.Polygon([[lon, lat] for lon, lat in bbox_list[36].get_polygon()])]}, crs=local_CRS)

In [ ]:
fig_size = 10
fig, ax = plt.subplots(figsize=(fig_size, fig_size))
zone_covered.plot(ax=ax, facecolor="w", edgecolor="b", alpha=0.1)
ax.imshow(small_patch.data_timeless["BANDS"], extent=(left, right, bottom, top))
small_box_gdf.plot(ax=ax, facecolor="w", edgecolor="r", alpha=0.1)
#bbox_gdf[bbox_gdf.index.isin(patch_ids)].plot(ax=ax, facecolor="w", edgecolor="r", alpha=0.1)
   

In [ ]:
plt.imshow(small_patch.data_timeless["BANDS"], extent=(left, right, bottom, top))

In [ ]:
bbox_list[patch_ids]

In [ ]:

patches = []
for bbox in bbox_list[patch_ids]:
    print(type(bbox))
    patch = create_subpatch(eopatch, bbox)
    patches.append(patch)
        

In [ ]:
# Draw the RGB images
fig, axs = plt.subplots(ncols=3, nrows=3, figsize=(20, 20))


for i in tqdm(range(len(patch_ids))):

    ax = axs[i // 3][i % 3]
    ax.imshow(patches[i].data_timeless["BANDS"])
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_aspect("auto")
    

fig.subplots_adjust(wspace=0, hspace=0)

In [ ]:
eopatch.bbox.lower_left

In [ ]:
small_patch = create_subpatch(eopatch, bbox_list[36])
small_patch

In [ ]:

def create_subpatch(eopatch, small_bbox):

    # Find the pixel indices corresponding to the small_bbox
    height, width = eopatch.data_timeless['BANDS'].shape[:2]

    min_x, min_y = small_bbox.lower_left
    max_x, max_y = small_bbox.upper_right
    print(min_x, min_y, max_x, max_y)
    patch_min_x, patch_min_y = eopatch.bbox.lower_left
    patch_max_x, patch_max_y = eopatch.bbox.upper_right
    # compute coord of each pixel of eopatch
    x_pxl_coord = np.linspace(patch_min_x, patch_max_x, width)
    y_pxl_coord = np.linspace(patch_min_y, patch_max_y, height)
    x_min_idx = np.searchsorted(x_pxl_coord, min_x)
    x_max_idx = np.searchsorted(x_pxl_coord, max_x)
    y_min_idx = np.searchsorted(y_pxl_coord, min_y)
    y_max_idx = np.searchsorted(y_pxl_coord, max_y)
    print(x_min_idx, x_max_idx, y_min_idx, y_max_idx)
    # Create a new EOPatch
    new_eopatch = EOPatch(bbox=BBox(bbox=(min_x, min_y, max_x, max_y), crs=local_CRS))

    # Copy data features
    for feature_type, feature_name in eopatch.get_features():
        print(feature_name)
        if feature_type.is_spatial():
            new_eopatch[feature_type][feature_name] = eopatch[feature_type][feature_name][height-y_max_idx:height-y_min_idx, x_min_idx:x_max_idx, :]

    return new_eopatch

# Using eo-learn

In [ ]:
from shapely.geometry import Polygon

from tqdm.auto import tqdm
import geopandas as gpd

from sentinelhub import DataCollection, UtmZoneSplitter

# Imports from eo-learn and sentinelhub
from eolearn.core import (
    EOExecutor,
    EOPatch,
    EOTask,
    EOWorkflow,
    FeatureType,
    LoadTask,
    MergeFeatureTask,
    OverwritePermission,
    SaveTask,
    linearly_connect_tasks,
)
from eolearn.features import NormalizedDifferenceIndexTask, SimpleFilterTask
from eolearn.features.extra.interpolation import LinearInterpolationTask
from eolearn.geometry import ErosionTask, VectorToRasterTask
from eolearn.io import ExportToTiffTask, SentinelHubInputTask, VectorImportTask
from eolearn.ml_tools import FractionSamplingTask

In [ ]:
# Folder where data for running the notebook is stored
DATA_FOLDER = os.path.join(datasets_dir, "RPG/PaysBasque")
# Locations for collected data and intermediate results
EOPATCH_FOLDER = os.path.join(".", "eopatches")
EOPATCH_SAMPLES_FOLDER = os.path.join(".", "eopatches_sampled")
RESULTS_FOLDER = os.path.join(".", "results")
for folder in (EOPATCH_FOLDER, EOPATCH_SAMPLES_FOLDER, RESULTS_FOLDER):
    os.makedirs(folder, exist_ok=True)
#TODO compute regional crs
local_CRS = '32630'
# Load geojson file
zone_covered = gpd.read_file(os.path.join(DATA_FOLDER, "contour-du-pays-basque.geojson")).to_crs(local_CRS)

# Add 500m buffer to secure sufficient data near border
#zone_covered = zone_covered.buffer(500)

# Get the country's shape in polygon format
zone_covered_shape = zone_covered.geometry.values[0]

# Plot country
zone_covered.plot()
plt.axis("off")

# Print size
zone_covered_width = zone_covered_shape.bounds[2] - zone_covered_shape.bounds[0]
zone_covered_height = zone_covered_shape.bounds[3] - zone_covered_shape.bounds[1]
print(f"Dimension of the area is {zone_covered_width:.0f} x {zone_covered_height:.0f} m2")

In [ ]:
zone_covered.bounds

In [ ]:
# Coordinates format
zone_covered.crs

In [ ]:
resolution = 10 # meters per pixel, used for raster dimension and satellite images
patch_dim = 255 # number of pixel

# Create a splitter to obtain a list of bboxes 
bbox_splitter = UtmZoneSplitter([zone_covered_shape], local_CRS, patch_dim*resolution)
bbox_list = np.array(bbox_splitter.get_bbox_list())
info_list = np.array(bbox_splitter.get_info_list())

# Prepare info of selected EOPatches
geometry = [Polygon(bbox.get_polygon()) for bbox in bbox_list]

idxs = [info["index"] for info in info_list]
idxs_x = [info["index_x"] for info in info_list]
idxs_y = [info["index_y"] for info in info_list]

bbox_gdf = gpd.GeoDataFrame({"index": idxs, "index_x": idxs_x, "index_y": idxs_y}, crs=local_CRS, geometry=geometry)

# Save to shapefile
shapefile_name = "grid_paysbasque_500x500.gpkg"
bbox_gdf.to_file(os.path.join(RESULTS_FOLDER, shapefile_name), driver="GPKG")

In [ ]:
size_surrounding = 1
size_of_selected_area = 2*size_surrounding + 1
# select a 5x5 area (id of center patch)
ID = 27

# Obtain surrounding 5x5 patches
patch_ids = []
for idx, info in enumerate(info_list):
    if abs(info["index_x"] - info_list[ID]["index_x"]) <= size_surrounding and abs(info["index_y"] - info_list[ID]["index_y"]) <= size_surrounding:
        patch_ids.append(idx)

# Check if final size is 5x5
if len(patch_ids) != size_of_selected_area * size_of_selected_area:
    print("Warning! Use a different central patch ID, this one is on the border.")

# Change the order of the patches (useful for plotting)
patch_ids = np.transpose(np.fliplr(np.array(patch_ids).reshape(size_of_selected_area, size_of_selected_area))).ravel()

# Save to shapefile
shapefile_name = "grid_paysbasque_500x500.gpkg"
bbox_gdf.to_file(os.path.join(RESULTS_FOLDER, shapefile_name), driver="GPKG")

In [ ]:
# Display bboxes over country

fig_size = 10
fig, ax = plt.subplots(figsize=(fig_size, fig_size))
ax.set_title(f"Selected {size_of_selected_area}x{size_of_selected_area} tiles from Pays basque", fontsize=0.2*fig_size)
zone_covered.plot(ax=ax, facecolor="w", edgecolor="b", alpha=0.5)
bbox_gdf.plot(ax=ax, facecolor="w", edgecolor="r", alpha=0.5)

for bbox, info in zip(bbox_list, info_list):
    geo = bbox.geometry
    ax.text(geo.centroid.x, geo.centroid.y, info["index"], ha="center", va="center")

# Mark bboxes of selected area
bbox_gdf[bbox_gdf.index.isin(patch_ids)].plot(ax=ax, facecolor="g", edgecolor="r", alpha=0.5)

plt.axis("off");

In [ ]:
add_data = SentinelHubInputTask(
    bands_feature=(FeatureType.DATA, "BANDS"),
    bands=band_names,
    resolution=10,
    maxcc=0.8,
    time_difference=datetime.timedelta(minutes=120),
    data_collection=DataCollection.SENTINEL2_L1C,
    additional_data=[(FeatureType.MASK, "dataMask", "IS_DATA"), (FeatureType.MASK, "CLM"), (FeatureType.DATA, "CLP")],
    max_threads=5,
)

### EOTasks definition 

In [ ]:
class SentinelHubValidDataTask(EOTask):
    """
    Combine Sen2Cor's classification map with `IS_DATA` to define a `VALID_DATA_SH` mask
    The SentinelHub's cloud mask is asumed to be found in eopatch.mask['CLM']
    """

    def __init__(self, output_feature):
        self.output_feature = output_feature

    def execute(self, eopatch):
        eopatch[self.output_feature] = eopatch.mask["IS_DATA"].astype(bool) & (~eopatch.mask["CLM"].astype(bool))
        return eopatch


class AddValidCountTask(EOTask):
    """
    The task counts number of valid observations in time-series and stores the results in the timeless mask.
    """

    def __init__(self, count_what, feature_name):
        self.what = count_what
        self.name = feature_name

    def execute(self, eopatch):
        eopatch[FeatureType.MASK_TIMELESS, self.name] = np.count_nonzero(eopatch.mask[self.what], axis=0)
        return eopatch

In [ ]:
# BAND DATA
# Add a request for S2 bands.
# Here we also do a simple filter of cloudy scenes (on tile level).
# The s2cloudless masks and probabilities are requested via additional data.
band_names = ["B02", "B03", "B04", "B08", "B11", "B12"]
add_data = SentinelHubInputTask(
    bands_feature=(FeatureType.DATA, "BANDS"),
    bands=band_names,
    resolution=10,
    maxcc=0.8,
    time_difference=datetime.timedelta(minutes=120),
    data_collection=DataCollection.SENTINEL2_L1C,
    additional_data=[(FeatureType.MASK, "dataMask", "IS_DATA"), (FeatureType.MASK, "CLM"), (FeatureType.DATA, "CLP")],
    max_threads=5,
)


# CALCULATING NEW FEATURES
# NDVI: (B08 - B04)/(B08 + B04)
# NDWI: (B03 - B08)/(B03 + B08)
# NDBI: (B11 - B08)/(B11 + B08)
ndvi = NormalizedDifferenceIndexTask(
    (FeatureType.DATA, "BANDS"), (FeatureType.DATA, "NDVI"), [band_names.index("B08"), band_names.index("B04")]
)
ndwi = NormalizedDifferenceIndexTask(
    (FeatureType.DATA, "BANDS"), (FeatureType.DATA, "NDWI"), [band_names.index("B03"), band_names.index("B08")]
)
ndbi = NormalizedDifferenceIndexTask(
    (FeatureType.DATA, "BANDS"), (FeatureType.DATA, "NDBI"), [band_names.index("B11"), band_names.index("B08")]
)


# VALIDITY MASK
# Validate pixels using SentinelHub's cloud detection mask and region of acquisition
add_sh_validmask = SentinelHubValidDataTask((FeatureType.MASK, "IS_VALID"))

# COUNTING VALID PIXELS
# Count the number of valid observations per pixel using valid data mask
add_valid_count = AddValidCountTask("IS_VALID", "VALID_COUNT")

In [ ]:
# SAVING TO OUTPUT (if needed)
save = SaveTask(EOPATCH_FOLDER, overwrite_permission=OverwritePermission.OVERWRITE_FEATURES)

### Proprocess data

In [ ]:
paysbasque_geojson_path = os.path.join(DATA_FOLDER, "rpg-2020-registre-parcellaire-graphique.geojson")
# change coordinates system of input data
paysbasque_geojson = gpd.read_file(paysbasque_geojson_path).to_crs('32630')
paysbasque_geojson.dropna(subset=['grp_cultu'], inplace=True)

In [ ]:
# OneHotEncode grp_cultu column in grp_encoded
encode_dict = {name: i for i, name in enumerate(paysbasque_geojson['grp_cultu'].unique())}
paysbasque_geojson['grp_encoded'] = paysbasque_geojson['grp_cultu'].replace(encode_dict)

In [ ]:
# Save preprocessed data
paysbasque_geopd_path = os.path.join(DATA_FOLDER, "rpg-2020-modified.geojson")
paysbasque_geojson.to_file(paysbasque_geopd_path)

In [ ]:
paysbasque_geojson = gpd.read_file(paysbasque_geopd_path)

In [ ]:
paysbasque_geojson.head()

### Create color map for display

In [ ]:
grp_cultu_list = list(paysbasque_geojson['grp_encoded'].unique())
cmap = plt.get_cmap('coolwarm', len(grp_cultu_list))
grp_cultu_dict = {}
for i, cultu in enumerate(grp_cultu_list):
    grp_cultu_dict[cultu]={
        "name": cultu,
        "id": i,
        "color": cmap(i)
    }

pb_norm = BoundaryNorm([x - 0.5 for x in range(len(grp_cultu_dict) + 1)], cmap.N)

In [ ]:
grp_cultu_list = list(paysbasque_geojson['grp_cultu'].unique())
grp_cultu_list = [cultu.replace(" ", "_").replace("(", "").replace(")", "") for cultu in grp_cultu_list if cultu is not None]
cmap = plt.get_cmap('viridis', len(grp_cultu_list))

RPG_colors_dict ={}    
      
for i, cultu in enumerate(grp_cultu_list):
    RPG_colors_dict[cultu]={"name": cultu, 
                            "id": i, 
                            "color": list_colors[i]
                           }

# Reference colormap things
RPG_norm = BoundaryNorm([x - 0.5 for x in range(RPG_colors_dict + 1)], cmap.N)

### RPG import part of the pipeline/workflow

In [ ]:
vector_feature = FeatureType.VECTOR_TIMELESS, "PB_REFERENCE"

vector_import_task = VectorImportTask(vector_feature, paysbasque_geopd_path)

rasterization_task = VectorToRasterTask(
    vector_feature,
    (FeatureType.MASK_TIMELESS, "pb"),
    values_column="grp_encoded",
    #raster_shape=(FeatureType.MASK, "IS_DATA"),
    raster_resolution=10,
    raster_dtype=np.uint8,
)

In [ ]:
paysbasque_geojson = os.path.join(DATA_FOLDER, "rpg-2020-registre-parcellaire-graphique.geojson")
vector_feature = FeatureType.VECTOR_TIMELESS, "RPG_REFERENCE"

vector_import_task = VectorImportTask(vector_feature, paysbasque_geojson)

rasterization_task = VectorToRasterTask(
    vector_feature,
    (FeatureType.MASK_TIMELESS, "RPG_colors_dict"),
    values_column="grp_cultu",
    #raster_shape=(FeatureType.MASK, "IS_DATA"),
    raster_resolution=10,
    raster_dtype=np.uint8,
)
# SAVING TO OUTPUT (if needed)
save = SaveTask(EOPATCH_FOLDER, overwrite_permission=OverwritePermission.OVERWRITE_FEATURES)

In [ ]:
# Define the workflow
workflow_nodes = linearly_connect_tasks(
    vector_import_task, rasterization_task, save
)
workflow = EOWorkflow(workflow_nodes)

# Let's visualize it
workflow.dependency_graph()

In [ ]:
# Define additional parameters of the workflow
input_node = workflow_nodes[0]
save_node = workflow_nodes[-1]
execution_args = []
for idx, bbox in enumerate(bbox_list[patch_ids]):
    execution_args.append(
        {
            input_node: {"bbox": bbox},
            save_node: {"eopatch_folder": f"eopatch_{idx}"},
        }
    )

# Execute the workflow
executor = EOExecutor(workflow, execution_args, save_logs=True)
executor.run(workers=4)

executor.make_report()

In [ ]:
fig, axs = plt.subplots(nrows=5, ncols=5, figsize=(20, 25))

for i in tqdm(range(len(patch_ids))):
    eopatch_path = os.path.join(EOPATCH_FOLDER, f"eopatch_{i}")
    eopatch = EOPatch.load(eopatch_path, lazy_loading=True)

    ax = axs[i // 5][i % 5]
    im = ax.imshow(eopatch.mask_timeless["RPG_colors_dict"].squeeze(), cmap=lulc_cmap, norm=lulc_norm)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_aspect("auto")
    del eopatch

fig.subplots_adjust(wspace=0, hspace=0)

cb = fig.colorbar(im, ax=axs.ravel().tolist(), orientation="horizontal", pad=0.01, aspect=100)
cb.ax.tick_params(labelsize=20)
cb.set_ticks([entry.id for entry in LULC])
cb.ax.set_xticklabels([entry.name for entry in LULC], rotation=45, fontsize=15)
plt.show();

In [ ]:
# Define the workflow
workflow_nodes = linearly_connect_tasks(
   vector_import_task, rasterization_task, save
 # add_data, ndvi, ndwi, ndbi, add_sh_validmask, add_valid_count, vector_import_task, rasterization_task, save

)
workflow = EOWorkflow(workflow_nodes)

# Let's visualize it
workflow.dependency_graph()

### Launch the workflow

In [ ]:
%%time

# Time interval for the SH request
time_interval = ["2019-01-01", "2019-12-31"]

# Define additional parameters of the workflow
input_node = workflow_nodes[0]
save_node = workflow_nodes[-1]
execution_args = []
for idx, bbox in enumerate(bbox_list[patch_ids]):
    execution_args.append(
        {
            input_node: {"bbox": bbox},
            save_node: {"eopatch_folder": f"eopatch_{idx}"},
        }
    )


# Execute the workflow
executor = EOExecutor(workflow, execution_args, save_logs=True)
executor.run(workers=4)

executor.make_report()

failed_ids = executor.get_failed_executions()
if failed_ids:
    raise RuntimeError(
        f"Execution failed EOPatches with IDs:\n{failed_ids}\n"
        f"For more info check report at {executor.get_report_path()}"
    )

In [ ]:
fig, axs = plt.subplots(nrows=3, ncols=3, figsize=(20, 25))

for i in tqdm(range(len(patch_ids))):
    eopatch_path = os.path.join(EOPATCH_FOLDER, f"eopatch_{i}")
    eopatch = EOPatch.load(eopatch_path, lazy_loading=True)

    ax = axs[i // 3][i % 3]
    im = ax.imshow(eopatch.mask_timeless["pb"].squeeze(), cmap=cmap, norm=pb_norm)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_aspect("auto")
    del eopatch

fig.subplots_adjust(wspace=0, hspace=0)

cb = fig.colorbar(im, ax=axs.ravel().tolist(), orientation="horizontal", pad=0.01, aspect=100)
cb.ax.tick_params(labelsize=20)
cb.set_ticks([grp_cultu_dict[key]['id'] for key in grp_cultu_dict.keys()])
cb.ax.set_xticklabels([grp_cultu_dict[key]['name'] for key in grp_cultu_dict.keys()], rotation=45, fontsize=15)
plt.show();